## Artifical Neural Networks

#### Importing Libraries

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf

#### Data Preprocessing

In [12]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

***We will eliminate the first two columns because they don't contain any relevant data***

In [13]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [14]:
print(y)

[1 0 1 ... 1 1 0]


#### Splitting the data into train & test

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#### Encoding the data

***We will use a label encoder for gender. We are using this because gender is a binary operation***

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
X_train[:, 2] = le.fit_transform(X_train[:, 2])
X_test[:, 2] = le.fit_transform(X_test[:, 2])

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.fit_transform(X_test))

In [19]:
print(X_train)

[[0.0 0.0 1.0 ... 1 0 163830.64]
 [0.0 1.0 0.0 ... 1 1 57098.0]
 [1.0 0.0 0.0 ... 1 0 185630.76]
 ...
 [1.0 0.0 0.0 ... 1 0 181429.87]
 [0.0 0.0 1.0 ... 1 1 148750.16]
 [0.0 1.0 0.0 ... 1 0 118855.26]]


In [20]:
print(X_test)

[[0.0 1.0 0.0 ... 1 1 192852.67]
 [1.0 0.0 0.0 ... 1 0 128702.1]
 [0.0 0.0 1.0 ... 1 1 75732.25]
 ...
 [0.0 0.0 1.0 ... 1 0 141533.19]
 [0.0 1.0 0.0 ... 1 1 11276.48]
 [0.0 1.0 0.0 ... 1 0 192950.6]]


***We want to use OneHotEncoding on the cateogorical variable geography***

#### Feature Scaling

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

In [23]:
ann = tf.keras.models.Sequential()

#### First input layer

In [24]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#### Second input layer

In [25]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#### Adding the output layer

In [26]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

***We made the activation sigmoid so we can get a probability function. We made the last unit once since we are dealing with a binary classification***

## Training the ANN

In [27]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ["accuracy"])

***Best optimizer that can perform stochastic gradient descent. This will update the weights to reduce the loss (error). We will use adam***

***For binary classification, we must use binary_crossentropy. If we had 3 or more, we would use categorical_crossentropy
We must also chnage the actication from sigmoid to softmax***

***For metrics, we will use accuracy***

In [28]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100) #32 is a typical value

Epoch 1/100
250/250 [==============================] - 1s 904us/step - loss: 0.6382 - accuracy: 0.6466
Epoch 2/100
250/250 [==============================] - 0s 828us/step - loss: 0.4910 - accuracy: 0.7991
Epoch 3/100
250/250 [==============================] - 0s 804us/step - loss: 0.4520 - accuracy: 0.8073
Epoch 4/100
250/250 [==============================] - 0s 813us/step - loss: 0.4351 - accuracy: 0.8163
Epoch 5/100
250/250 [==============================] - 0s 767us/step - loss: 0.4251 - accuracy: 0.8223
Epoch 6/100
250/250 [==============================] - 0s 783us/step - loss: 0.4185 - accuracy: 0.8234
Epoch 7/100
250/250 [==============================] - 0s 774us/step - loss: 0.4135 - accuracy: 0.8250
Epoch 8/100
250/250 [==============================] - 0s 789us/step - loss: 0.4091 - accuracy: 0.8279
Epoch 9/100
250/250 [==============================] - 0s 828us/step - loss: 0.4058 - accuracy: 0.8303
Epoch 10/100
250/250 [==============================] - 0s 795us/step - l

#### Lets try one example

***A customer came in with the following details: Geography = France, 
Credit score = 600, Gender = Male, Age = 40, Tenure = 3 years, Balance = 60000, Number of Products = 2, Has credit card = yes, Active member = yes, estimated salary = 50000, We want to predict whether they will leave or not?***

In [29]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

1/1 [==============================] - 0s 81ms/step
[[0.02865002]]


***There is a 3% chance that this customer will leave the bank. We will predict that the customer will stay with the bank***

#### Now lets do the test results

In [30]:
pred = ann.predict(X_test)
pred = (pred > 0.5)
from sklearn import metrics
print(metrics.classification_report(y_test, pred))

63/63 [==============================] - 0s 727us/step
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      1595
           1       0.73      0.51      0.60       405

    accuracy                           0.86      2000
   macro avg       0.81      0.73      0.76      2000
weighted avg       0.85      0.86      0.85      2000



In [31]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred)
print(cm)

[[1519   76]
 [ 199  206]]


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn import model_selection

In [33]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC()))
models.append(('LSVC', LinearSVC()))
models.append(('RFC', RandomForestClassifier()))
models.append(('DTR', DecisionTreeRegressor()))

In [34]:
seed = 7
results = []
names = []
X = X_train
Y = y_train
import warnings 
warnings.filterwarnings("ignore")
for name, model in models:
    kfold = model_selection.KFold( n_splits=10, shuffle=True,random_state=42)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (
        name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.807000 (0.011635)
KNN: 0.825000 (0.011208)
NB: 0.816750 (0.009862)
SVC: 0.855000 (0.014208)
LSVC: 0.808750 (0.010592)
RFC: 0.861000 (0.013332)
DTR: 0.784000 (0.015164)


***ANN seems to have the best accuracy, we will use ANN model for determing future cusotmers churn rate***

### Lets create a pipeline for the data

In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [43]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
X = dataset.iloc[:,3:-1]
y = dataset.iloc[:,-1]

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [50]:
myModel = clf.fit(X_train, y_train)

In [51]:
myModel

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(handle_unknown='ignore')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['CreditScore', 'Age',
                                                   'Balance',
                                                   'EstimatedSalary']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                    

In [52]:
pred = myModel.predict(X_test)

In [55]:
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.866
